# Forecast

Based on our current scouting, as well as The Blue Alliance's OPR score, and any other criteria we have, guess the outcome of the matches to be played, and display the matches
that have already been played using the same criteria

*run all Data notebooks before any of the analysis notebooks.*

In [1]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn,  HTMLTemplateFormatter
from bokeh.io import output_notebook, show

import glob
import logging

import numpy as np
import pandas as pd

import sys

from common import canalytics

output_notebook()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('Forecast_Analysis')

Loading BokehJS ...

In [2]:
# Without index_col=0, pandas will create a new, made up index for each line read from the csv.
tba_matches = pd.read_csv('./data/tba_matches.csv',index_col=0)
team_summary = pd.read_csv('./data/team_summary.csv',index_col=0)
team_summary.set_index('team',inplace=True)


In [3]:
team_summary.head()


,scouted_matches,matches,valid_tba_and_scouting,scouting_points_total,climb_success_total,climb_failed_total,climb_points_total,teleop_cargo_upper_total,teleop_cargo_lower_total,auto_cargo_upper_total,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
team,,,,,,,,,,,,,,,,,,,,,
418,7,11,7,91.0,9,0,94,3.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,8,12,8,47.0,8,2,57,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1164,7,12,7,14.0,4,1,33,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2158,6,12,6,156.0,7,2,78,44.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2613,9,12,9,50.0,0,0,0,0.0,38.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
tba_matches.head()

,match_number,post_result_time,tvalid,predicted_time,red1,red2,red3,red1_taxi,red2_taxi,red3_taxi,...,blue1_climb,blue2_climb,blue3_climb,blue_points,blue_fouls,blue_techfouls,blue_teleop_cargo,blue_auto_cargo,blue_teleop_low,blue_teleop_high
0,1,2022-04-01 11:09,T,2022-04-01 11:05,8625,457,2158,Yes,Yes,Yes,...,None,High,Mid,35,0,0,6,1,1,5
11,2,2022-04-01 11:19,T,2022-04-01 11:16,7534,6682,8591,Yes,No,No,...,Mid,None,None,24,0,0,6,0,0,6
22,3,2022-04-01 18:50,T,2022-04-01 11:27,7540,8580,8408,Yes,Yes,Yes,...,None,None,None,65,1,0,27,3,9,18
33,4,2022-04-01 11:39,T,2022-04-01 11:34,418,8325,4153,Yes,No,Yes,...,None,None,Mid,28,0,0,12,2,12,0
44,5,2022-04-01 13:07,T,2022-04-01 13:04,8507,4734,7492,Yes,Yes,Yes,...,None,None,Mid,61,1,0,18,3,1,17


In [5]:

tba_matches['oprs_forecast_red']=tba_matches.apply(lambda row: team_summary.at[row.red1,'oprs']+team_summary.at[row.red2,'oprs']+team_summary.at[row.red3,'oprs'], axis = 1 )
tba_matches['oprs_forecast_blue']=tba_matches.apply(lambda row: team_summary.at[row.blue1,'oprs']+team_summary.at[row.blue2,'oprs']+team_summary.at[row.blue3,'oprs'], axis = 1 )
tba_matches['scouting_forecast_red']=tba_matches.apply(lambda row: team_summary.at[row.red1,'scouting_points_avg']+
                                                       team_summary.at[row.red2,'scouting_points_avg']+
                                                       team_summary.at[row.red3,'scouting_points_avg'], axis = 1 )
tba_matches['scouting_forecast_blue']=tba_matches.apply(lambda row: team_summary.at[row.blue1,'scouting_points_avg']+
                                                        team_summary.at[row.blue2,'scouting_points_avg']+
                                                        team_summary.at[row.blue3,'scouting_points_avg'], axis = 1 )
# Remove all columns outside of what we are using to put in the table.
tba_matches=tba_matches[['match_number','predicted_time','post_result_time','red1','red2','red3','blue1','blue2','blue3',
                         'red_points','blue_points','oprs_forecast_red','oprs_forecast_blue','scouting_forecast_red','scouting_forecast_blue']]

# This rounds across all columns and rows in the dataframe, and then sorts the dataframe to ensure it is in match order.
tba_matches=tba_matches.round()
tba_matches.sort_values(by='match_number',inplace=True)

In [6]:
source = ColumnDataSource(tba_matches)

redgocans_template="""                
            <p style="color:<%= 
                  (function redcolorfromteam(){
                    if (red1 == 2158 || red2 == 2158 || red3 == 2158 ){
                      return('#FFCF46')}
                    }()) %>;
                background-color:<%=
                  (function redbackgroundfromteam(){
                    if (team == 2158 ){
                      return('black')}
                    }()) %>;
                    "> 
                <%= value %>
            </p>
"""
redteam_formatter =  HTMLTemplateFormatter(template=redgocans_template)


bluegocans_template="""                
            <p style="color:<%= 
                  (function bluecolorfromteam(){
                    if (blue1 == 2158 || blue2 == 2158 || blue3 == 2158 ){
                      return('#FFCF46')}
                    }()) %>;
                background-color:<%=
                  (function bluebackgroundfromteam(){
                    if (team == 2158 ){
                      return('black')}
                    }()) %>;
                    "> 
                <%= value %>
            </p>
"""
blueteam_formatter =  HTMLTemplateFormatter(template=bluegocans_template)

columns = [
        TableColumn(field="match_number", title="Match"),
        TableColumn(field="predicted_time", title="Time", width=700),
        TableColumn(field="red1", title="Red 1"),
        TableColumn(field="red2", title="Red 2"),
        TableColumn(field="red3", title="Red 3"),
        TableColumn(field="blue1", title="Blue 1"),
        TableColumn(field="blue2", title="Blue 2"),
        TableColumn(field="blue3", title="Blue 3"),
        TableColumn(field="red_points", title="Red Actual"),
        TableColumn(field="blue_points", title="Blue Actual"),
        TableColumn(field="oprs_forecast_red", title="OPRS Red"),
        TableColumn(field="oprs_forecast_blue", title="OPRS Blue"),
        TableColumn(field="scouting_forecast_red", title="Scout Red"),
        TableColumn(field="scouting_forecast_blue", title="Scout Blue")]
    

    
forecast_table = DataTable(source=source, columns=columns, width=1000,height=450,
        sortable=True,
        editable=False,
        fit_columns=True,
        reorderable=True,
        frozen_rows=0)

# Forecasting 

The table shows all of the current qualifying matches for the event.  If the event has already occurred, it will show the Actual score for both Red and Blue teams.

In addition it will also show two forecasts for each match.  One is based on OPRS from The Blue Alliance, and the other is average points based on the scouting app input.




In [7]:
show(forecast_table)